In [1]:
import pandas as pd
import numpy as np
import torch
import random
from transformers import BertTokenizer, BertModel

In [2]:
!pip install nltk

In [2]:
random_seed = 19
random.seed(random_seed)
torch.manual_seed(random_seed)

In [31]:
pd.set_option('display.max_colwidth', None)

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [4]:
model = BertModel.from_pretrained("bert-base-uncased")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
# obtain BERT embedding for the entire review of the training dataset
# this will allow the training of the linear model of the mapping
# of BERT to OpenAI embeddings

review_embedding_train = []

for i in range(len(train)):
  text = train.iloc[i,2]
  encoded_input = tokenizer(text, truncation=True, max_length=512, return_tensors='pt')
  output = model(**encoded_input)
  review_token_embedding = output.last_hidden_state
  review_embedding = review_token_embedding.mean(dim=1)
  review_embedding_test.append(review_embedding.detach().numpy())

In [ ]:
a = np.asarray(review_embedding_train)

In [ ]:
np.save('bert_train.npy', a)

In [46]:
# obtain sentence-wise embeddings for the 10 randomly selected
# test reviews

import nltk
from nltk import sent_tokenize
nltk.download('punkt_tab')

test_index_positive = [29761, 40204, 15336, 20775, 10708] # for random selection see main JupyterNotebook file
test_index_negative = [31448, 47821,  3206, 16660, 38123] # for random selection see main JupyterNotebook file

review_embedding_test_positive = []
review_embedding_test_negative = []

# positive reviews
for i in test_index_positive:
  embedding_individual_review_positive = []
  text = test[test['id']==i].iloc[0,1]
  text = text.replace("<br /><br />", " ") # clean text, which often contains '<br /><br />' to improve tokenization
  sentences = sent_tokenize(text) # split each review into sentences

  for sentence in sentences:
    encoded_input = tokenizer(sentence, truncation=True, max_length=512, return_tensors='pt')
    output = model(**encoded_input)
    review_token_embedding = output.last_hidden_state
    review_embedding = review_token_embedding.mean(dim=1) # obtain embedding for entire sentence
    embedding_individual_review_positive.append(review_embedding.detach().numpy())
  review_embedding_test_positive.append(np.asarray(embedding_individual_review_positive))

# negative reviews
for i in test_index_negative:
  embedding_individual_review_negative = []
  text = test[test['id']==i].iloc[0,1]
  text = text.replace("<br /><br />", " ") # clean text, which often contains '<br /><br />' to improve tokenization
  sentences = sent_tokenize(text) # split each review into sentences

  for sentence in sentences:
    encoded_input = tokenizer(sentence, truncation=True, max_length=512, return_tensors='pt')
    output = model(**encoded_input)
    review_token_embedding = output.last_hidden_state
    review_embedding = review_token_embedding.mean(dim=1) # obtain embedding for entire sentence
    embedding_individual_review_negative.append(review_embedding.detach().numpy())
  review_embedding_test_negative.append(np.asarray(embedding_individual_review_negative))


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [47]:
# save sentence embeddings for random positive and negative reviews
import pickle

# save
with open('review_embedding_test_positive.pkl','wb') as f:
    pickle.dump(review_embedding_test_positive,f)

with open('review_embedding_test_negative.pkl','wb') as f:
    pickle.dump(review_embedding_test_negative,f)

Train linear regression

In [57]:
# load BERT embeddings for training data
bert_review_train = np.load('bert_train.npy').squeeze()

In [58]:
# load OpenAI embeddings for training data
openai_train = train.iloc[:, 3:]

In [61]:
# align BERT and OpenAI embeddings using the BERT embeddings
# representing the entire review as the OpenAI embedding does the same
from sklearn.linear_model import LinearRegression

align = LinearRegression()
align.fit(bert_review_train, openai_train)

LinearRegression()

In [65]:
# save linear regression model
import pickle

# save
with open('align.pkl','wb') as f:
    pickle.dump(align,f)